# Getting Started

## Install the SingleStore package

The SingleStore package can be installed the following way:
```
pip install singlestoredb
```

In addition, you can install the SQLAlchemy and Ibis plugins with the following:
```
pip install singlestore[dataframe]
```

## Import SingleStore

In [1]:
import singlestoredb as s2

## Create a connection

This function is compliant with the Python DB-API 2.0. In addition, it allows you to use a URL
as a connection string rather than individual parameters. Parameters may also be set using
environment variable (e.g., `SINGLESTOREDB_HOST`, `SINGLESTOREDB_USER`, `SINGLESTOREDB_PASSWORD`, etc.),
but it is easiest to specify the entire URL in the `SINGLESTOREDB_URL` environment variable.

In [2]:
conn_url = 'root:@127.0.0.1:9306/x_db'

In [3]:
conn = s2.connect(conn_url)

The URL in the code above can take a couple of forms. The default driver is `mysql`, but you can also specify `http` for connecting to the SingleStoreDB data API.
```
http://root:@localhost:9000/x_db
```

### Environment Variables

Connection URLs can also be set using an environment variable. This allows you to create environments that already have connection parameters
embedded in them so that new connections can be made without specifying any parameters.

In [4]:
import os

os.environ['SINGLESTOREDB_URL'] = conn_url

### Cursors

Most interaction with databases are done using cursors. To create a cursor from a connection, you use the `cursor` method.

In [5]:
cur = conn.cursor()

## Create tables and insert data

Using the cursor, we can execute commands for creating tables and inserting data.

### Reset Tables

In [6]:
cur.execute(r'drop table if exists departments')
cur.execute(r'drop table if exists employees')
cur.execute(r'drop table if exists salaries')

cur.execute(r'''
create table if not exists departments (
  id int,
  name varchar(255),
  primary key (id)
);''')

cur.execute(r'''
create table if not exists employees (
  id int,
  deptId int,
  managerId int,
  name varchar(255),
  hireDate date,
  state char(2),
  primary key (id)
);''')

cur.execute(r'''
create table if not exists salaries (
  employeeId int,
  salary int,
  primary key (employeeId)
);''')

0

### Insert Data Using Named Parameters

Using named parameters such as `:foo` is the default parameter format in queries. This requires dictionaries for the parameter structure.

In [7]:
cur.executemany(r'insert into departments(id, name) values (%(id)s, %(name)s)',
    [
        dict(id=1, name='Marketing'),
        dict(id=2, name='Finance'),
        dict(id=3, name='Sales'),
        dict(id=4, name='Customer Service'),
    ]
)

4

### Insert Data Using Positional Parameters

You can also use positional parameters which specify the one-based index in a list or tuple.

In [8]:
cur.executemany(r'insert into employees (id, deptId, managerId, name, hireDate, state) '
                r'values (%s, %s, %s, %s, %s, %s)',
    [
        (1, 2, None, "Karly Steele", "2011-08-25", "NY"),
        (2, 1, 1, "Rhona Nichols", "2008-09-11", "TX"),
        (3, 4, 2, "Hedda Kent", "2005-10-27", "TX"),
        (4, 2, 1, "Orli Strong", "2001-07-01", "NY"),
        (5, 1, 1, "Leonard Haynes", "2011-05-30", "MS"),
        (6, 1, 5, "Colette Payne", "2002-10-22", "MS"),
        (7, 3, 4, "Cooper Hatfield", "2010-08-19", "NY"),
        (8, 2, 4, "Timothy Battle", "2001-01-21", "NY"),
        (9, 3, 1, "Doris Munoz", "2008-10-22", "NY"),
        (10, 4, 2, "Alea Wiggins", "2007-08-21", "TX"),
    ]
)

10

### Insert Data From DataFrame

In addition to this method, you can use the `pandas.DataFrame.to_sql` method with a SQLAlchemy SingleStore connection object.

In [75]:
import pandas as pd

df = pd.DataFrame( 
    [
        (1, 885219), 
        (2, 451519), 
        (3, 288905), 
        (4, 904312), 
        (5, 919124),
        (6, 101538), 
        (7, 355077), 
        (8, 900436), 
        (9, 41557), 
        (10, 556263),
    ],
    columns=['employeeId', 'salary'])

cur.executemany(r'insert into salaries (employeeId, salary) '
                r'values (%s, %s)', df)

10

### Insert Data with load_data
For large datasets we can leverage optimized SingleStore LOAD DATA statement.

In [9]:
import fastavro
import io

new_employees_records = [
    {'id': 11, 'deptId': 2, 'managerId': 1, 'name': "Leonard Karly", 'hireDate': "2012-08-25", 'state': "CA"},
    {'id': 12, 'deptId': 1, 'managerId': 1, 'name': "Colette Rhona", 'hireDate': "2009-09-11", 'state': "CA"},
    {'id': 13, 'deptId': 4, 'managerId': 1, 'name': "Kent Hatfield", 'hireDate': "2006-10-27", 'state': "CA"},
]

schema = {
    'name': 'employee',
    'type': 'record',
    'fields': [
        {'name': 'id', 'type': 'int'},
        {'name': 'deptId', 'type': 'int'},
        {'name': 'managerId', 'type': 'int'},
        {'name': 'name', 'type': 'string'},
        {'name': 'hireDate', 'type': 'string'},
        {'name': 'state', 'type': 'string'},
    ],
}
parsed_schema = fastavro.parse_schema(schema)

buf = io.BytesIO()
fastavro.writer(buf, parsed_schema, new_employees_records)

load_data_stmt = """
    LOAD DATA LOCAL INFILE 'placeholder' 
    INTO TABLE employees
    FORMAT AVRO
    (id <- %::id, deptId <- %::deptId, managerId <- %::managerId, name <- %::name, hireDate <- %::hireDate, state <- %::state)
"""

cur.load_data(query=load_data_stmt, data_input_stream=buf)

3

### Queries to compare INSERT and LOAD DATA performance

In [15]:
import random
import time

n_records = 10 ** 6

In [16]:
shift = int(random.getrandbits(30))

lot_of_records = [{'id': 11, 'deptId': 2, 'managerId': 1, 'name': "perf test", 'hireDate': "2012-12-20", 'state': "CA"} for _ in range(n_records)]
for idx in range(n_records):
    lot_of_records[idx]['id'] = idx + shift

start_time = time.time()
cur.executemany(r'insert into employees (id, deptId, managerId, name, hireDate, state) '
                r'values (%(id)s, %(deptId)s, %(managerId)s, %(name)s, %(hireDate)s, %(state)s)',
    lot_of_records
)
print("--- %s seconds executemany ---" % (time.time() - start_time))

--- 21.428183794021606 seconds executemany ---


In [18]:

shift = int(random.getrandbits(30))

lot_of_records_2 = [{'id': 11, 'deptId': 2, 'managerId': 1, 'name': "perf test", 'hireDate': "2012-12-20", 'state': "CA"} for _ in range(n_records)]
for idx in range(len(lot_of_records_2)):
    lot_of_records_2[idx]['id'] = idx + shift

start_time = time.time()
buf = io.BytesIO()
fastavro.writer(buf, parsed_schema, lot_of_records_2)
print("--- %s seconds for fastavro.writer ---" % (time.time() - start_time))

load_data_stmt = """
    LOAD DATA LOCAL INFILE 'placeholder' 
    INTO TABLE employees
    FORMAT AVRO
    (id <- %::id, deptId <- %::deptId, managerId <- %::managerId, name <- %::name, hireDate <- %::hireDate, state <- %::state)
"""

cur.load_data(query=load_data_stmt, data_input_stream=buf)
print("--- %s seconds load_data ---" % (time.time() - start_time))

--- 1.302696704864502 seconds for fastavro.writer ---
--- 9.594840288162231 seconds load_data ---


# Querying data

Here we are executing a simple `SELECT` operation.

In [10]:
cur.execute('select name from employees')

10

There are various ways of fetching data including the `fetchone()`, `fetchmany()`, and `fetchall()` methods
description in the DB-API. In addition, you can simply iterate over the cursor itself.

In [11]:
for item in cur:
    print(item)

('Cooper Hatfield',)
('Rhona Nichols',)
('Orli Strong',)
('Karly Steele',)
('Hedda Kent',)
('Doris Munoz',)
('Leonard Haynes',)
('Colette Payne',)
('Timothy Battle',)
('Alea Wiggins',)


By default, results come back in tuples, but there are other result structure options as well, they include `namedtuple` and `dict`.

In [12]:
conn = s2.connect(conn_url, results_type='dicts')

In [13]:
cur = conn.cursor()
cur.execute('select name from employees')
cur.fetchall()

[{'name': 'Cooper Hatfield'},
 {'name': 'Rhona Nichols'},
 {'name': 'Orli Strong'},
 {'name': 'Karly Steele'},
 {'name': 'Hedda Kent'},
 {'name': 'Doris Munoz'},
 {'name': 'Leonard Haynes'},
 {'name': 'Colette Payne'},
 {'name': 'Timothy Battle'},
 {'name': 'Alea Wiggins'}]

The following is a query using parameter substitution. While the underlying connection object is a `mysql.connector`-based
object which takes the pyformat parameter format, we are using the `sqlparams` package to allow us to 
use whatever substitution form we want. Since the HTTP API uses question marks for parameters, we have
adopted that format here as well.

In [14]:
cur.execute('select name, hireDate from employees where name like %s', ['%Rhona%'])

1

The `description` field returns information about the query results. It includes fields such as `name`, `type_code`,
`display_size`, `internal_size`, etc. These are defined in the DB-API, but are not all populated.

In [15]:
cur.description

(Description(name='name', type_code=253, display_size=None, internal_size=255, precision=255, scale=0, null_ok=True, flags=0, charset=33),
 Description(name='hireDate', type_code=10, display_size=None, internal_size=30, precision=30, scale=31, null_ok=True, flags=0, charset=33))

In [16]:
cur.fetchall()

[{'name': 'Rhona Nichols', 'hireDate': datetime.date(2008, 9, 11)}]

## Server Variables

Server variables can be accessed through a number of dictionary-like members on the connection. The members are `globals`, `locals`, `cluster_globals`, `cluster_locals` when a scope is specified, or `vars` and `cluster_vars` for all variables regardless of scope.

In [17]:
dict(conn.globals)

{'active_query_users_blacklist': '',
 'activities_delta_sleep_s': '1',
 'advanced_hdfs_pipelines': False,
 'aes_default_encryption_mode': 'aes-128-ecb',
 'aggregator_failure_detection': False,
 'allow_async_bottomless': False,
 'allow_ipv6': False,
 'allow_newer_to_older_replication': False,
 'allow_proc_cpuinfo_errors': False,
 'assert_on_cluster_db_reprovisioning': False,
 'async_backup': True,
 'auditlog_disk_sync': False,
 'auditlog_level': False,
 'auditlog_retention_period': '0',
 'auditlog_rotation_size': '134217728',
 'auditlog_rotation_time': '3600',
 'auto_replicate': False,
 'autocommit': True,
 'autostats_flush_interval_secs': '600',
 'backup_max_threads': '0',
 'backup_multipart_upload_concurrency': '3',
 'basedir': '/src/memsql/release',
 'batch_external_functions': 'ALWAYS',
 'batch_external_functions_size': '512',
 'blob_cache_eviction_policy': 'LRU2',
 'blob_cache_threadpool_max_concurrent': '32',
 'bottomless_blob_compression_level': '0',
 'bottomless_compression_leve

In [18]:
conn.globals.enable_external_functions = True

In [19]:
conn.globals.enable_external_functions

True

In [20]:
cur.execute('show variables like "enable_external_functions"')
cur.fetchall()

[{'Variable_name': 'enable_external_functions', 'Value': 'ON'}]

## Enabling the HTTP API

The HTTP API can be enabled using the `conn.enable_http_api` method. This method can optionally set the port number.
If a port number is not specified, the existing setting will be used. The port number is returned.

In [21]:
conn.enable_http_api(port=8100)

8100

## Create an HTTP connection

In [22]:
s2.options.results.type = 'namedtuples'

In [23]:
http_conn = s2.connect('http://root:@localhost:8100/x_db')

In [24]:
http_cur = http_conn.cursor()

## Query data using the HTTP connection

In [25]:
http_cur.execute('select name from employees')

10

In [26]:
http_cur.description

[Description(name='name', type_code=253, display_size=None, internal_size=None, precision=None, scale=None, null_ok=True, flags=0, charset=0)]

In [27]:
http_cur.fetchall()

[Row(name='Leonard Haynes'),
 Row(name='Colette Payne'),
 Row(name='Timothy Battle'),
 Row(name='Alea Wiggins'),
 Row(name='Cooper Hatfield'),
 Row(name='Rhona Nichols'),
 Row(name='Orli Strong'),
 Row(name='Karly Steele'),
 Row(name='Hedda Kent'),
 Row(name='Doris Munoz')]

In [28]:
http_cur.execute('select name, hireDate from employees where name like %s', ['%Rhona%'])

1

In [29]:
http_cur.description

[Description(name='name', type_code=253, display_size=None, internal_size=None, precision=None, scale=None, null_ok=True, flags=0, charset=0),
 Description(name='hireDate', type_code=10, display_size=None, internal_size=None, precision=None, scale=None, null_ok=True, flags=0, charset=0)]

In [30]:
df = http_cur.fetchall()
df

[Row(name='Rhona Nichols', hireDate=datetime.date(2008, 9, 11))]

## Workspace Management

The objects allow you to manage workspaces and create database connections to those workspaces. In order for this call
to work, you either need to pass in a cluster management API token or have one set in your `SINGLESTOREDB_MANAGEMENT_TOKEN` environment variable.

In [31]:
wm = s2.manage_workspaces()

### Get Current Clusters and Regions

In [32]:
wm.workspace_groups

[WorkspaceGroup(name='Python Client Testing', id='17b0fa56-bb7f-48d4-b351-f6296505711d', created_at=datetime.datetime(2023, 9, 26, 0, 0), firewall_ranges=['0.0.0.0/0'], region=Region(name='US East 1 (N. Virginia) - HD2', id='452cc4b1-df20-4130-9e2f-e72ba79e3d46', provider='AWS')),
 WorkspaceGroup(name='Group 2', id='24fe2eca-af66-45c3-918b-035780eb177a', created_at=datetime.datetime(2023, 10, 16, 0, 0), firewall_ranges=['166.113.65.66/32'], region=Region(name='US West 2 (Oregon)', id='64031b39-3da1-4a7b-8d3d-6ca86e8d71a7', provider='AWS'))]

In [33]:
wm.regions

[Region(name='Europe West 2 (London)', id='04eb4250-5417-4300-9822-70cf4f114543', provider='AWS'),
 Region(name='Asia Southeast 1 (Singapore)', id='0bbbde94-7c33-4592-95bc-470332396a82', provider='GCP'),
 Region(name='US West 1 (Oregon)', id='1c1de314-2cc0-4c74-bd54-5047ff90842e', provider='GCP'),
 Region(name='Europe West 2 (London)', id='1cbf85d7-0ae2-49ab-ad09-35fd19c02f04', provider='GCP'),
 Region(name='US Central 1 (Iowa)', id='22a913e1-ace5-4362-afa0-3d21a01d6d1c', provider='GCP'),
 Region(name='Asia South 1 (Mumbai)', id='3186a27b-87d5-4b84-a93b-90c57fd2be74', provider='GCP'),
 Region(name='US East 1 (N. Virginia)', id='3482219c-a389-4079-b18b-d50662524e8a', provider='AWS'),
 Region(name='Asia Pacific Southeast 1 (Singapore)', id='3d226d4b-90b4-4a8d-848c-6c8403ee905c', provider='AWS'),
 Region(name='US East 1 (N. Virginia) - HD2', id='452cc4b1-df20-4130-9e2f-e72ba79e3d46', provider='AWS'),
 Region(name='Europe Central 1 (Frankfurt)', id='5781fbe5-8e2d-40ee-8cfe-82d6cfe74aed', p

Objects returned by `WorkspaceManager.workspace_groups`, `WorkspaceManager.regions`, 
and `WorkspaceGroup.workspaces` can be indexed by name and ID as well.

In [34]:
reg = wm.regions['US West 1 (Oregon)']
reg

Region(name='US West 1 (Oregon)', id='1c1de314-2cc0-4c74-bd54-5047ff90842e', provider='GCP')

In [35]:
reg = wm.regions['3d226d4b-90b4-4a8d-848c-6c8403ee905c']
reg

Region(name='Asia Pacific Southeast 1 (Singapore)', id='3d226d4b-90b4-4a8d-848c-6c8403ee905c', provider='AWS')

### Create a Workspace Group

In [36]:
import secrets
password = secrets.token_urlsafe(20)

In [37]:
wg = wm.create_workspace_group(
    'Demo Workspace Group', 
    region=[x for x in wm.regions if x.name.startswith('US')][0],
    admin_password=password, 
    firewall_ranges=['0.0.0.0/0'],
)
wg

WorkspaceGroup(name='Demo Workspace Group', id='b877f081-25cc-4c4c-8552-b96a919bd0aa', created_at=datetime.datetime(2023, 10, 17, 0, 0), region=Region(name='US West 1 (Oregon)', id='1c1de314-2cc0-4c74-bd54-5047ff90842e', provider='GCP'))

### Create a Workspace in the Group

In [38]:
ws = wg.create_workspace('workspace-1', wait_on_active=True)

In [39]:
ws

Workspace(name='workspace-1', id='670492f6-c458-4b5d-88e6-1bf6a8d07add', created_at=datetime.datetime(2023, 10, 17, 0, 0), endpoint='svc-b877f081-25cc-4c4c-88e6-1bf6a8d07add-dml.gcp-oregon-1.svc.singlestore.com', group_id='b877f081-25cc-4c4c-8552-b96a919bd0aa', size='S-00', state='ACTIVE')

### Connect to the Workspace

In [40]:
with ws.connect(user='admin', password=password) as conn:
    with conn.cursor() as cur:
        cur.execute('show databases')
        print(cur.fetchall())

[Row(Database='cluster'), Row(Database='information_schema'), Row(Database='memsql')]


In [41]:
ws.terminate(wait_on_terminated=True)

### Working with Stages

Stages is a place when you can store data and SQL files for use in your database. Stage files can 
be written to interactively, or be used as files by other Python objects. Stages are
created on a per-workspacegroup basis, so we'll use the workspace group created above.

Currently there are no files in our Stage.

In [42]:
wg.stages.listdir()

[]

#### Uploading a file

The most basic usage of stages is to upload files from disk. We will upload a local file named
`test.sql` to the `stage_test.sql` file.

In [43]:
f = wg.stages.upload_file('test.sql', 'stage_test.sql')
f

StagesObject(name='stage_test.sql', created_at=datetime.datetime(2023, 10, 17, 0, 0), format='text', last_modified_at=datetime.datetime(2023, 10, 17, 0, 0), mimetype='text/plain', path=PurePosixPath('stage_test.sql'), size=88, type='file', writable=True)

In [44]:
wg.stages.listdir()

['stage_test.sql']

Information about the file can be accessed using the resulting object's attributes and methods.

In [45]:
print('name', f.name)
print('created at', f.created_at)
print('is writable?', f.writable)
print('is dir?', f.is_dir())

name stage_test.sql
created at 2023-10-17 00:00:00
is writable? True
is dir? False


#### Downloading content

The file contents can also be downloaded. Note that the returned content will by in bytes unless
you specify an `encoding=` parameter.

In [46]:
print(f.download(encoding='utf-8'))

CREATE TABLE people (
    id BIGINT,
    first_name CHAR(30),
    last_name CHAR(50)
);



#### Using the file interface

It is also possible to treat a Stage file as a local file using Python's file interface.
Note that this creates a temporary in-memory file until the final read / write is completed,
then the data is synced.

In [47]:
with wg.stages.open('stage_open_test.csv', 'w') as wfile:
    wfile.write('A,B,C\n')
    wfile.write('1,2,3\n')
    wfile.write('4,5,6\n')

In [48]:
print(wg.stages.download('stage_open_test.csv', encoding='utf-8'))

A,B,C
1,2,3
4,5,6



Since the `open` method returns an object that supports Python's file interface, we can use
it from other APIs as well such as pandas' `read_csv` function.

In [49]:
import pandas as pd

with wg.stages.open('stage_open_test.csv', 'r') as rfile:
    df = pd.read_csv(rfile)
    
df

,A,B,C
0,1,2,3
1,4,5,6


In [50]:
with wg.stages.open('stage_pandas_test.csv', 'w') as wfile:
    df.to_csv(wfile, index=False)
    
print(wg.stages.download('stage_pandas_test.csv', encoding='utf-8'))

A,B,C
1,2,3
4,5,6



## Clean up

In [51]:
wg.terminate()